In [3]:
import hddm
import pandas as pd
import pickle
import numpy as np
import os
import ssms.basic_simulators 
from ssms.basic_simulators import simulator as sim

In [8]:
import pandas as pd
import ssms.basic_simulators 
from ssms.basic_simulators import simulator as sim

def simulate_subject_data(subjectIndex, test, modelParamsDF, num_simulations=100):
    """
    Simulates data for a given subject.

    Parameters:
    subjectIndex (int): The index of the subject.
    test (pd.DataFrame): The DataFrame containing test data.
    modelParamsDF (pd.DataFrame): The DataFrame containing model parameters.
    num_simulations (int): The number of simulations per trial.

    Returns:
    pd.DataFrame: A DataFrame containing the simulated data.
    """
    # Load parameters for the subject
    v_intercept = modelParamsDF.loc['v_Intercept_subj.{}'.format(subjectIndex), 'mean']
    # v_gain = modelParamsDF.loc['v_I(gain + loss)_subj.{}'.format(subjectIndex), 'mean'] # for no lambda models 
    v_gain = modelParamsDF.loc['v_gain_subj.{}'.format(subjectIndex), 'mean']
    # v_loss = modelParamsDF.loc['v_I(gain + loss)_subj.{}'.format(subjectIndex), 'mean'] # for no lambda models 
    v_loss = modelParamsDF.loc['v_loss_subj.{}'.format(subjectIndex), 'mean']
    a = modelParamsDF.loc['a_subj.{}'.format(subjectIndex), 'mean']
    z = modelParamsDF.loc['z_subj.{}'.format(subjectIndex), 'mean']
    # z = 0.5 # for no gamma models
    t = modelParamsDF.loc['t_subj.{}'.format(subjectIndex), 'mean']
    
    # Filter test data for the current subject
    subject_data = test[test['subj_idx'] == subjectIndex]
    
    simulated_rts = []
    simulated_choices = []
    
    # Simulate for each trial of the subject
    for idx, row in subject_data.iterrows():
        v = v_intercept + v_gain * row['gain'] + v_loss * row['loss']
        # v = v_gain * row['gain'] + v_loss * row['loss'] # for no alpha models
        trial_data = sim.simulator(model='ddm', theta=[v, a, z, t], n_samples=num_simulations)
        
        for rt in trial_data['rts']:
            simulated_rts.append(rt[0])
        for choice in trial_data['choices']:
            simulated_choices.append(choice[0])

    # Convert -1 choices to 0
    simulated_choices = [1 if choice == 1 else 0 for choice in simulated_choices]

    # Replicate the subject and trial data to match the number of simulations
    subj_idx_replicated = [subjectIndex] * len(simulated_rts)
    gain_replicated = np.repeat(subject_data['gain'].values, num_simulations)
    loss_replicated = np.repeat(subject_data['loss'].values, num_simulations)

    simulated_df = pd.DataFrame({
        'subj_idx': subj_idx_replicated,
        'response': simulated_choices,
        'rt': simulated_rts,
        'gain': gain_replicated,
        'loss': loss_replicated
    })
    
    return simulated_df


In [20]:
modelName = 'no_gamma_ej5'
m = hddm.load('savedModels/{}/{}'.format(modelName, modelName))
modelParamsDF = m.nodes_db
test = pd.read_csv('j5.csv')


total_subjects = 49
num_simulations_per_trial = 100

# Initialize an empty DataFrame to hold all simulated data
all_simulated_data = pd.DataFrame()

for subject_index in range(1, total_subjects + 1):
    simulated_df = simulate_subject_data(subject_index, test, modelParamsDF, num_simulations=num_simulations_per_trial)
    
    # Append the simulated data for this subject to the main DataFrame
    all_simulated_data = pd.concat([all_simulated_data, simulated_df], ignore_index=True)

# Once all subjects are processed, save the combined DataFrame to CSV
all_simulated_data.to_csv('sim_j5_no_gamma.csv', index=False)

In [66]:
modelName = 'no_alpha_ej1'
m = hddm.load('savedModels/{}/{}'.format(modelName, modelName))
m.dic

13052.964936655608

In [54]:
df = pd.read_csv('ej1.csv')
df

,subj_idx,leftV,rightV,keyPress,up_accept,response,rt,gain,loss,gPlusLoss
0,1,-60,70,-1,-1,1,8.310281,70,-60,10
1,1,-70,100,-1,-1,1,5.077738,100,-70,30
2,1,100,-70,1,1,1,1.136604,100,-70,30
3,1,60,-30,-1,-1,1,1.301045,60,-30,30
4,1,-30,60,-1,-1,1,0.798285,60,-30,30
...,...,...,...,...,...,...,...,...,...,...
7149,49,30,-60,-1,1,0,0.804447,30,-60,-30
7150,49,90,-60,-1,-1,1,1.851457,90,-60,30
7151,49,-60,90,1,1,1,5.667128,90,-60,30
7152,49,-70,50,-1,1,0,1.270307,50,-70,-20


In [65]:
a=2.199472
v_loss = 0.026332
v_gain = 0.017186
v_intercept = -0.038661
z = 0.413934
t = 0.442387
v = v_intercept + v_gain * 50 + v_loss * -50
simulated_rts = []
simulated_choices = []
trial_data = sim.simulator(model='ddm', theta=[v, a, z, t], n_samples=100)
for rt in trial_data['rts']:
    simulated_rts.append(rt[0])
for choices in trial_data['choices']:
    simulated_choices.append(choices[0])
    
    
simulated_rts
simulated_choices

[-1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1]

In [41]:
model = 'angle'
n_subjects = 1
n_samples_by_subject = 500
hddm.simulators.hddm_dataset_generators
from hddm.simulators.hddm_dataset_generators import simulator_h_c as sim
sim

data, full_parameter_dict = sim(n_subjects = n_subjects,n_samples_by_subject = n_samples_by_subject,
                                                                                  model = model,
                                                                                  p_outlier = 0.00,
                                                                                  conditions = None,
                                                                                  depends_on = None,
                                                                                  regression_models = None,
                                                                                  regression_covariates = None,
                                                                                  group_only_regressors = False,
                                                                                  group_only = None,
                                                                                  fixed_at_default = None)

TypeError: 'module' object is not callable

NameError: name 'simulator_h_c' is not defined